In [2]:
import numpy as np
import pandas as pd
import glob
from sklearn import linear_model, metrics
import scipy.optimize
import random
from math import exp
from math import log
import random

In [3]:
df = pd.read_csv('data/train.csv', index_col=None, header=0)
df_test = pd.read_csv('data/test.csv', index_col=None, header=0)

In [14]:
### HOT ENCODING ###
for i in range(0,9):
  if i==7:
    continue
  featureName = 'X'+str(i)
  for c in df[featureName].unique():
    df[featureName+c] = np.where(df[featureName]==c, 1, 0)
  df.drop[featureName]

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X8q,X8w,X8g,X8y,X8l,X8f,X8u,X8r,X8t,X8c
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,0,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,0,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,0,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0
5,18,92.93,t,b,e,c,d,g,h,s,...,0,0,0,0,0,0,0,0,0,0
6,24,128.76,al,r,e,f,d,f,h,s,...,0,0,0,0,0,0,0,0,0,0
7,25,91.91,o,l,as,f,d,f,j,a,...,0,0,0,0,0,0,0,0,0,0
8,27,108.67,w,s,as,e,d,f,i,h,...,0,0,0,0,0,0,0,0,0,0
9,30,126.99,j,b,aq,c,d,f,a,e,...,0,0,0,0,0,0,0,0,0,0
